In [1]:
import torch 
import json
import cv2
import yaml
import sys
import re
import csv
import numpy as np
import os

sys.path.append('../Configs/')

from configs import update_configs, get_configs

/home/mihnea/anaconda3/envs/vqa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_cfg_file = "../Configs/dataset.yaml"
model_cfg_file = "../Configs/model.yaml"
train_cfg_file = "../Configs/train.yaml"
labels_cfg_file = "../Configs/labels.yaml"


In [3]:
cfg = get_configs()
update_configs(cfg, model_cfg_file, dataset_cfg_file, train_cfg_file)

In [4]:
with open(cfg.DATASET.TRAIN_FILE) as f:
    d = json.load(f)

# TODO some statistics about the data

In [5]:
answers = []
for keys, values in d.items():
    answers.extend(values['answers'])

answers = set(answers)
labels = {index : answer for index, answer in enumerate(answers)}

with open(labels_cfg_file) as yaml_file:
    yaml.dump({'ANSWERS_LABELS': labels}, yaml_file, default_flow_style=False)

UnsupportedOperation: not writable

In [6]:
words_list = []
for keys, values in d.items():
    question = values['question']
    question = re.sub(r'[^\w\s]', '', question.lower())
    words = question.split()
    words_list.extend(words)

print(len(set(words_list)))

4475


In [8]:
for keys, values in d.items():
   # print(keys)
   pass
   

In [9]:
with open("file.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        print(row)

FileNotFoundError: [Errno 2] No such file or directory: 'file.tsv'

In [10]:
key = list(d)[4]
values = d[key]

print(values)

{'question': 'Does the horse look tired?', 'image_id': '464980', 'image_name': 'COCO_train2014_000000464980.jpg', 'answers': ['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']}


In [17]:
data = np.load(os.path.join(cfg.DATASET.TRAIN_VAL_IMG_EMBEDDINGS_FOLDER, '9.npz'))
print(data['feat'].shape)

(36, 2048)
